In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
file_config = {
    'train': '',
    'test': '',
}
params = {
    'code': 'code'
}


In [ ]:
data = pd.read_parquet(file_config['train'])
tdata = pd.read_parquet(file_config['test'])
total = pd.concat([data, tdata]).reset_index(drop=True)

In [ ]:
filteringDict = {
    'deltaAge': 36,
    'numSep': 10,
    'unique_leng':10,
#     'tfidf-threshold':1.5,
    'months': 12
}

In [ ]:
total['numSep'] = total[params['code']].apply(lambda x: len(np.where(x=='SEP')[0]))
print('begining: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

# keep patient with number of visits larger than numSep
total = total[(total.numSep>filteringDict['numSep'])]
total = total.reset_index(drop=True)
print('filtering by visit: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

# keep patient with records having time span larger than deltaAge
total['deltaAge'] = total.age.apply(lambda x: int(x[-1]) - int(x[0]))
total = total[total['deltaAge'] >filteringDict['deltaAge']]
total = total.reset_index(drop=True)
print('filtering by time span: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

# keep patient with number of unique code larger than unique_leng
total['unique_leng'] = total['code'].apply(lambda x: len(np.unique(x)))
total = total[total['unique_leng'] >filteringDict['unique_leng']]
total = total.reset_index(drop=True)
print('filtering by number of unique codes: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

In [ ]:
# split
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(total, test_size=0.3, random_state=42)
X_train.to_parquet('')
X_test.to_parquet('')